In [64]:
import pandas as pd
import numpy as np

In [4]:
%ls

 Volume in drive C is Windows-SSD
 Volume Serial Number is 220E-4420

 Directory of c:\Users\lenha\Repos\fantasy-baseball-draft\CBS_fantasy_baseball

03/05/2021  07:30 AM    <DIR>          .
03/05/2021  07:30 AM    <DIR>          ..
03/05/2021  07:30 AM                 0 __init__.py
03/05/2021  07:30 AM               290 assistant.py
03/05/2021  07:30 AM            21,785 BaseFantasyClasses.py
03/26/2021  07:42 AM            23,621 development_notebook.ipynb
03/05/2021  07:30 AM    <DIR>          file_handlers
03/05/2021  07:30 AM             1,011 pitchers.py
03/05/2021  07:30 AM    <DIR>          player
03/05/2021  07:30 AM            13,513 stats.py
03/05/2021  07:30 AM    <DIR>          team
               6 File(s)         60,220 bytes
               5 Dir(s)  77,711,114,240 bytes free


In [8]:
df1 = pd.read_csv('../data/projections/2021/bat hitters.csv')
df2 = pd.read_csv('../data/projections/2021/bat x hitters.csv')
df3 = pd.read_csv('../data/projections/2021/steamer batters.csv')
df4 = pd.read_csv('../data/projections/2021/zips hitters.csv')

In [27]:
dfs = [df1, df2, df3, df4]
score_cats = ['R','HR', 'RBI', 'H', 'AB', 'SB']
bio_cats = ['Name', 'playerid','Team','PA']
for df in dfs:
    for c in score_cats:
        if c not in df.columns:
            print('Ahk!')
    for c in bio_cats:
        if c not in df.columns:
            print("Yeesh!")

In [39]:
def df_to_fb_stats(df):
    if df['HR'].dtype:
        pass
    df = df[bio_cats + score_cats].copy()
    for col in score_cats:
        df[col] = df[col]/df['PA']
    return df
dfs2 = []
for df in dfs:
    df = df_to_fb_stats(df)
    dfs2.append(df)
dfs2[0].head()
dfs = dfs2

In [73]:
merged.select_dtypes(object).columns

Index(['Name_bat', 'playerid', 'Team_bat', 'Name_xbat', 'Team_xbat',
       'Name_stmr', 'Team_stmr', 'Name_zips', 'Team_zips', 'Name'],
      dtype='object')

In [42]:
merged = None
merged = pd.merge(dfs[0], dfs[1], 'outer', left_on='playerid', right_on='playerid', suffixes=('_bat', '_xbat'))
merged = pd.merge(merged, dfs[2], 'outer', left_on='playerid', right_on='playerid')
merged = pd.merge(merged, dfs[3], 'outer', left_on='playerid', right_on='playerid', suffixes=('_stmr', '_zips'))

In [83]:
merged.isna()
#def agg_score_cols(df):
cols = merged.columns.str.startswith('Team')
cols = merged.columns[cols]
def set_team(series):
    if series.isna().all():
        return np.nan
    else:
        return series.dropna().iloc[0]
merged['Team'] = merged[cols].apply(set_team, axis = 1)

merged['Team'] = merged[cols].apply(lambda x: x.dropna().iloc[0], axis=1)
merged.iloc[0].dropna()
for stat in score_cats:
    cols = merged.columns.str.startswith(stat)
    merged[stat] = merged[merged.columns[cols]].apply(np.mean, axis = 1)


In [87]:
merged = merged[bio_cats[:-1] + score_cats]

In [92]:
merged.head()


,Name,playerid,Team,R,HR,RBI,H,AB,SB
0,Mike Trout,10155,LAA,0.168681,0.063104,0.167421,0.135713,0.795689,0.016909
1,Juan Soto,20123,WSN,0.172518,0.055979,0.177381,0.140262,0.819014,0.018970
2,Freddie Freeman,5361,ATL,0.155080,0.045781,0.156940,0.137330,0.849179,0.009746
3,Christian Yelich,11477,MIL,0.149860,0.051534,0.144707,0.133858,0.846531,0.027127
4,Bryce Harper,11579,PHI,0.157180,0.054264,0.159416,0.125204,0.814496,0.020378


In [94]:
pt = pd.read_csv('../data/projections/2021/fg_depth hitters.csv')
merged = pd.merge(merged, pt[['PA','playerid']], how='outer', left_on='playerid', right_on='playerid')
merged.head()

,Name,playerid,Team,R,HR,RBI,H,AB,SB,PA
0,Mike Trout,10155,LAA,0.168681,0.063104,0.167421,0.135713,0.795689,0.016909,679.0
1,Juan Soto,20123,WSN,0.172518,0.055979,0.177381,0.140262,0.819014,0.018970,665.0
2,Freddie Freeman,5361,ATL,0.155080,0.045781,0.156940,0.137330,0.849179,0.009746,679.0
3,Christian Yelich,11477,MIL,0.149860,0.051534,0.144707,0.133858,0.846531,0.027127,672.0
4,Bryce Harper,11579,PHI,0.157180,0.054264,0.159416,0.125204,0.814496,0.020378,665.0


In [128]:
#pd.read_csv('../data/projections/2021/cbs hitters.csv', skiprows=1, skipfooter=1)
id_map = pd.read_csv('../data/projections/2021/id_map.txt', engine='python', sep='\t')
#pd.read_csv?
#drop = id_map.columns.where(id_map.columns.str.startswith('Unnamed')).dropna()
#id_map.drop(drop, inplace=True, axis=1)
id_map.head()

,mlb_id,mlb_name,mlb_pos,mlb_team,mlb_team_long,bats,throws,birth_date,bp_id,bref_id,...,retro_name,debut,yahoo_id,yahoo_name,ottoneu_id,ottoneu_name,ottoneu_pos,rotowire_id,rotowire_name,rotowire_pos
0,592091,A.J. Achter,P,LAA,Los Angeles Angels,R,R,8/27/1988,66919.0,achteaj01,...,A.J. Achter,9/3/2014,9824.0,A.J. Achter,18093.0,A.J. Achter,RP,NaN,NaN,NaN
1,595918,A.J. Cole,P,TOR,Toronto Blue Jays,R,R,1/5/1992,68086.0,coleaj01,...,A.J. Cole,4/28/2015,9638.0,A.J. Cole,14940.0,A.J. Cole,RP,11446.0,A.J. Cole,P
2,454560,A.J. Ellis,C,SD,San Diego Padres,R,R,4/9/1981,47564.0,ellisaj01,...,A.J. Ellis,9/15/2008,8373.0,A.J. Ellis,1543.0,A.J. Ellis,Util,9242.0,A.J. Ellis,C
3,456167,A.J. Griffin,P,TEX,Texas Rangers,R,R,1/28/1988,65757.0,griffaj01,...,A.J. Griffin,6/24/2012,9220.0,A.J. Griffin,13764.0,A.J. Griffin,RP,12410.0,A.J. Griffin,P
4,543362,A.J. Jimenez,C,TEX,Texas Rangers,R,R,5/1/1990,58320.0,jimenaj01,...,A.J. Jimenez,9/6/2017,NaN,NaN,17777.0,A.J. Jimenez,Util,12071.0,A.J. Jimenez,C


In [118]:
id_map[['cbs_id',
       'cbs_name', 'cbs_pos', 'espn_id', 'espn_name', 'espn_pos', 'fg_id',
       'fg_name', 'fg_pos']]

,cbs_id,cbs_name,cbs_pos,espn_id,espn_name,espn_pos,fg_id,fg_name,fg_pos
0,66919,achteaj01,A.J.,Achter,2146207,A.J.,Achter,RP,33637
1,68086,coleaj01,A.J.,Cole,1839916,A.J.,Cole,RP,31595
2,47564,ellisaj01,A.J.,Ellis,1104367,A.J.,Ellis,C,29258
3,griffaj01,A.J.,Griffin,1974327,A.J.,Griffin,SP,32559,A.J.
4,jimenaj01,A.J.,Jimenez,1798970,A.J.,Jimenez,C,31050,A.J.
...,...,...,...,...,...,...,...,...,...
3302,wheelza01,Zack,Wheeler,1744777,Zack,Wheeler,SP,31267,Zack
3303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3304,58696,spruize01,Zeke,Spruill,1941064,Zeke,Spruill,SP,31633
3305,56964,wheelze01,Zelous,Wheeler,1601827,Zelous,Wheeler,3B,30249


In [ ]:
class FantasyValuator:
    '''A class for providing Standings Points Gained based valuations of hitters.

    Attributes
    ----------
    spg: Pandas Series
        A Pandas series of SPG values.

    league_BA: float
        The expected batting average the fantasy league in which the player's value will be evaluated.         Used for caluculating xH.

    league_AB: numeric
        The number of at bats of the fantasy team the player will play for.

    league_ERA: float
        The expected ERA of the fantasy league in which the player's value will be evaluated

    ###Docs a work in progress.
    '''
    def __init__():
        self.spg = pd.Series()
        self.spg['xH'] = 8.475##used 8.47, nearly matched the real value of 8.486 in 2019.
        self.spg['HR'] = 5.286##real value was 4.782
        self.spg['R'] = 11.671###used 10.81,real value was 12.532
        self.spg['RBI'] = 13.38##used 12.20,was 14.564
        self.spg['SB'] = 3.46##used 4.27,was 2.657

        self.spg['xER'] = 8.47# was in 2018, 7.85
        self.spg['SO'] = 17.47
        self.spg['W'] = 1.73
        self.spg['SV'] = 4.55
        self.spgx['WHIP'] = 12.420
        ###was 10.60 in 2018
        ##Shit, used 10.848 in 2019; actual value in 2019 was 13.992!
        
        
        self.league_BA = .265
        self.league_ERA = 3.79
        self.league_WHIP = 1.23
        self.hitter_replacement_level = 16.72
        self.pitcher_replacement_level = 11.44
        self.catcher_replacement_level = 9##This is an approximation.

        self.hitter_categories = ['HR','xH','RBI','R','SB']
        self.pitcher_categories = ['W','SV','SO','xWHIP','xER']

    def evaluate_hitters(self, hitters, use_replacement_level=True):
        '''
        Return hitter SPG based fantasy value from fantasy stats. 
        
        Paramters
        ---------
        hitters: dict-like or dataframe
            A dataframe or dictionary with columns or indices corresponding to the hitter stats to be
            evaluated in the fantasy wins-above-replacement calculation.

        Returns
        -------
            A pd.Series indexed to hitters (for DataFrame of hitters) or a float (for a dict-like of               hitters)
        '''
        eval_data = hitters[self.hitter_categories]
        fWAR = (eval_data/spg[self.hitter_categories]).sum()
        if use_replacement_level:
            fWAR = fWAR.mask('C' in hitters['elig'], other = fWAR - self.catcher_replacement_level)
            fWAR = fWAR.where('C' in hitters['elig'], other = fWAR - self.hitter_replacement_level)
        return fWAR

    def ERA_to_xER(self,pitchers):
        '''Gets a pitchers xER.'''
        return (pitchers['ERA']-self.league_ERA)*pitchers['IP']

    def BA_to_xH(self, hitters):
        '''Get hitters xH'''
        return (hitters['BA']-self.league_BA)*hitters['AB']

    def WHIP_to_xWHIP(self, pitchers):
        return (pitchers['WHIP']-self.league_WHIP)*pitchers['IP']

    def _calculate_fWAR(self, hitter_series):

        
        
        

In [5]:
import os
#os.listdir('../data')
test = pd.read_csv('../data/TestFiles/hittersProj.csv')


In [6]:
test.head()

,Name,Team,G,PA,AB,H,2B,3B,HR,R,...,CS,AVG,OBP,SLG,OPS,wOBA,Fld,BsR,WAR,playerid
0,Mike Trout,Angels,157,658,516,155,29,4,39,119,...,7,0.300,0.441,0.600,1.041,0.431,-0.6,3.4,9.3,10155
1,J.D. Martinez,Red Sox,155,651,574,170,35,2,39,101,...,2,0.297,0.372,0.569,0.940,0.393,-0.2,-2.5,4.1,6184
2,Bryce Harper,NaN,150,630,510,136,28,1,34,95,...,5,0.267,0.399,0.528,0.927,0.390,-2.6,0.7,4.9,11579
3,Mookie Betts,Red Sox,162,679,588,178,42,3,29,116,...,8,0.303,0.385,0.537,0.922,0.390,14.6,5.6,7.4,13611
4,Giancarlo Stanton,Yankees,145,609,530,142,25,1,45,95,...,1,0.267,0.354,0.570,0.923,0.386,0.8,-0.4,4.3,4949


In [20]:
x = pd.DataFrame({'elig':[['C','U'],['U']]})
x

,elig
0,"[C, U]"
1,[U]


In [21]:
x['catcher'] = False
x['catcher'] = x.mask('C' in x['elig'], True)
'C' in x['elig']

ValueError: Array conditional must be same shape as self